<a href="https://colab.research.google.com/github/asim5800/Deep-learning-practicals/blob/main/CovidCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:

# IMPORT NECESSARY LIBRARIES
import librosa
import librosa.display

import os, glob, sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline

from sklearn.model_selection import train_test_split

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip3 install pydub

In [ ]:
audio_df=pd.read_csv('/content/drive/MyDrive/clinical/labels.csv')

In [ ]:
audio_df

,date,corona_test,age,gender,medical_history,smoker,patient_reported_symptoms,cough_filename
0,4/21,negative,53,male,"none,",yes,"none,",neg-0421-083-cough-m-53.mp3
1,4/21,positive,50,male,"Congestive heart failure,",no,"Shortness of breath,",pos-0421-084-cough-m-50.mp3
2,4/21,negative,43,male,"none,",no,"Sore throat,",neg-0421-085-cough-m-43.mp3
3,4/21,positive,65,male,"Asthma or chronic lung disease,",no,"Shortness of breath,New or worsening cough,",pos-0421-086-cough-m-65.mp3
4,4/21,positive,40,female,"none,",no,"Sore throat,Loss of taste,Loss of smell,",pos-0421-087-cough-f-40.mp3
5,4/21,negative,66,female,"Diabetes with complications,",no,"none,",neg-0421-088-cough-f-66.mp3
6,4/21,negative,20,female,"none,",no,"none,",neg-0421-089-cough-f-20.mp3
7,4/21,negative,17,female,"none,",no,"Shortness of breath,Sore throat,Body aches,",neg-0421-090-cough-f-17.mp3
8,4/21,negative,47,male,"none,",no,"New or worsening cough,",neg-0421-091-cough-m-47.mp3
9,4/21,positive,53,male,"none,",no,"Fever, chills, or sweating,Shortness of breath...",pos-0421-092-cough-m-53.mp3


In [ ]:

import os
from pydub import AudioSegment

path = "/content/drive/MyDrive/clinical/segme"

#Change working directory
os.chdir(path)

audio_files = os.listdir()

# You dont need the number of files in the folder, just iterate over them directly using:
for file in audio_files:
    #spliting the file into the name and the extension
    name, ext = os.path.splitext(file)
    if ext == ".mp3":
       mp3_sound = AudioSegment.from_mp3(file)
       #rename them using the old name + ".wav"
       mp3_sound.export("{0}.wav".format(name), format="wav")

In [ ]:

import time
import os
path = '/content/drive/MyDrive/clinical/segmented/wav cough'
lst = []

start_time = time.time()

for subdir, dirs, files in os.walk(path):
  for file in files:
      try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(file[7:8]) - 1 
        arr = mfccs, file
        lst.append(arr)
      # If the file is not valid, skip it
      except ValueError:
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

--- Data loaded. Loading time: 24.894986152648926 seconds ---


In [ ]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)

In [ ]:

import numpy as np
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((121, 40), (121,))

In [ ]:
# Saving joblib files to not load them again with the loop above

import joblib

X_name = 'X.joblib'
y_name = 'y.joblib'
save_dir = '/content/drive/MyDrive/clinical/segmented/wav cough'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [ ]:

# Loading saved models
import joblib
X = joblib.load('/content/drive/MyDrive/clinical/segmented/wav cough/X.joblib')
y = joblib.load('/content/drive/MyDrive/clinical/segmented/wav cough/y.joblib')

In [ ]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [ ]:
# importing and initilizing random forest classifier
from sklearn.ensemble import RandomForestClassifier


rfc = RandomForestClassifier(n_estimators=10)
training_start = time.perf_counter()
rfc.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = rfc.predict(X_test)
prediction_end = time.perf_counter()
acc_rfc = (preds == y_test).sum().astype(float) / len(preds)*100
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start
print("Scikit-Learn's Random Forest Classifier's prediction accuracy is: %3.2f" % (acc_rfc))
print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

Scikit-Learn's Random Forest Classifier's prediction accuracy is: 85.00
Time consumed for training: 0.018 seconds
Time consumed for prediction: 0.00144 seconds


In [ ]:
#printing classification report and confusion matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92        34
           1       0.50      0.17      0.25         6

    accuracy                           0.85        40
   macro avg       0.68      0.57      0.58        40
weighted avg       0.81      0.85      0.82        40



In [ ]:
import numpy as np
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import optimizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical

from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(64, 5,padding='same',
                 input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(256, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 40, 64)            384       
_________________________________________________________________
activation (Activation)      (None, 40, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 40, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 128)           41088     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=260, validation_data=(x_testcnn, y_test))

Epoch 1/260
6/6 [==============================] - 1s 66ms/step - loss: 1.7898 - accuracy: 0.6173 - val_loss: 0.7566 - val_accuracy: 0.8500
Epoch 2/260
6/6 [==============================] - 0s 18ms/step - loss: 1.5845 - accuracy: 0.6543 - val_loss: 1.0407 - val_accuracy: 0.8500
Epoch 3/260
6/6 [==============================] - 0s 19ms/step - loss: 2.0597 - accuracy: 0.6667 - val_loss: 1.0150 - val_accuracy: 0.8500
Epoch 4/260
6/6 [==============================] - 0s 24ms/step - loss: 2.0797 - accuracy: 0.6914 - val_loss: 0.4268 - val_accuracy: 0.8500
Epoch 5/260
6/6 [==============================] - 0s 19ms/step - loss: 2.1065 - accuracy: 0.5185 - val_loss: 0.4707 - val_accuracy: 0.8500
Epoch 6/260
6/6 [==============================] - 0s 20ms/step - loss: 1.4549 - accuracy: 0.6049 - val_loss: 1.2676 - val_accuracy: 0.8500
Epoch 7/260
6/6 [==============================] - 0s 18ms/step - loss: 1.6032 - accuracy: 0.7407 - val_loss: 0.6724 - val_accuracy: 0.8500
Epoch 8/260
6/6 [===

In [ ]:
predictions=model.predict(x_testcnn)
predictions = np.argmax(predictions,axis=1)


In [ ]:
predictions
y_test

new_Ytest = y_test.astype(int)
new_Ytest

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(new_Ytest, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.94      0.93        34
           1       0.60      0.50      0.55         6

    accuracy                           0.88        40
   macro avg       0.76      0.72      0.74        40
weighted avg       0.87      0.88      0.87        40



In [ ]:
tf.keras.models.save_model(model,'/content/drive/MyDrive/clinical/segmented/wav cough/model.hdf5')
print("MODEL SAVED")

MODEL SAVED


In [ ]:
"""
This file can be used to try a live prediction. 
"""

import keras
import numpy as np
import librosa

class livePredictions:
    """
    Main class of the application.
    """

    def __init__(self, path, file):
        """
        Init method is used to initialize the main parameters.
        """
        self.path = path
        self.file = file

    def load_model(self):
        """
        Method to load the chosen model.
        :param path: path to your h5 model.
        :return: summary of the model with the .summary() function.
        """
        self.loaded_model = keras.models.load_model(self.path)
        return self.loaded_model.summary()

    def makepredictions(self):
        """
        Method to process the files and create your features.
        """
        data, sampling_rate = librosa.load(self.file)
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
        x = np.expand_dims(mfccs, axis=1)
        x = np.expand_dims(x, axis=0)
        predictions=self.loaded_model.predict(x)
        predictions = np.argmax(predictions,axis=1)
        #predictions = self.loaded_model.predict_classes(x)
        print("Prediction is", " ", self.convertclasstoemotion(predictions))

    @staticmethod
    def convertclasstoemotion(pred):
        """
        Method to convert the predictions (int) into human readable strings.
        """
        
        label_conversion = {'0': 'negative',
                            '1': 'positive'
                            }

        for key, value in label_conversion.items():
            if int(key) == pred:
                label = value
        return label

# Here you can replace path and file with the path of your model and of the file 
#from the RAVDESS dataset you want to use for the prediction,
# Below, I have used a neutral file: the prediction made is neutral.

pred = livePredictions(path='/content/drive/MyDrive/clinical/segmented/wav cough/model.hdf5',file='/content/drive/MyDrive/clinical/segmented/wav cough/pos-0421-093-cough-f-24-0.wav')

pred.load_model()
pred.makepredictions()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 40, 64)            384       
_________________________________________________________________
activation (Activation)      (None, 40, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 40, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 128)           41088     
_________________________________________________________________
activation_1 (Activation)    (None, 10, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0

#***THIS MODEL NEEDS IMPROVEMENT BEACUASE OF DATA SCARCITY.***